In [1]:
import os
import xml.etree.ElementTree as ET
from transformers import GPT2Tokenizer
import regex
import pickle

/home/c3ilab/Documents/Suryansh/MithilaVerse/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_all_files(directory):
    all_texts = []
    for file_name in os.listdir(directory):
        if file_name.endswith(".xml"):
            file_path = os.path.join(directory, file_name)
            all_texts.extend(load_single_file(file_path))
        if file_name.endswith(".txt"):
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                text_list = text.split('।')
                text_list = [segment.strip() for segment in text_list]
                all_texts.extend(text_list)
    return all_texts

In [3]:
def load_single_file(file_path):
    texts = []
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Navigate to the <body> and <p> tags
        body = root.find(".//body")
        if body:
            for paragraph in body.findall(".//p"):
                if paragraph.text:
                    texts.append(paragraph.text.strip())
    except ET.ParseError as e:
        print(f"Error parsing {file_path}: {e}")
    return texts

In [4]:
def clean_text(text):
    # Replacing multiple spaces with one
    text = regex.sub(r"\s+", " ", text)
    # Keep Devanagari and spaces
    text = regex.sub(r"[^\p{Devanagari}\s]", "", text)
    # Convert to lowercase
    text = text.lower().strip()
    return text

def clean_dataset(dataset):
    return [clean_text(text) for text in dataset]

In [5]:
from datasets import Dataset

In [6]:
if __name__ == "__main__":
    directory = "Maithili"
    
    raw_texts = load_all_files(directory)

    cleaned_texts = clean_dataset(raw_texts)

    print("First cleaned sample:", cleaned_texts[0])

First cleaned sample: ताराशंकर बंधोपाध्यायक जन्म  जुलाई  ईमे पश्चिम बंगालक बीरभूम जिलाक लाभपुर नामक गाममे भेल छल हुनक पिताक नाम श्री हरिदास बंधोपाध्याय और माताक नाम श्रीमती प्रभावती देवी छल ओ परिवारमे सभसँ पैघ छलाह हुनका एकटा बहिन और दूटाभाय छलनि


In [7]:
data = Dataset.from_dict({'text': cleaned_texts})

In [8]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the padding token to be the same as eos_token if it doesn't exist
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the cleaned dataset
def tokenize_function(examples):
    # Tokenize inputs and generate labels by shifting the input
    encodings = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    encodings['labels'] = encodings['input_ids'].copy()  # Copy the input_ids to labels
    return encodings
tokenized_data = data.map(tokenize_function, batched=True)

/home/c3ilab/Documents/Suryansh/MithilaVerse/env/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 53951/53951 [00:10<00:00, 5163.23 examples/s]


In [9]:
print("First tokenized sample:", tokenized_data[0])

First tokenized sample: {'text': 'ताराशंकर बंधोपाध्यायक जन्म  जुलाई  ईमे पश्चिम बंगालक बीरभूम जिलाक लाभपुर नामक गाममे भेल छल हुनक पिताक नाम श्री हरिदास बंधोपाध्याय और माताक नाम श्रीमती प्रभावती देवी छल ओ परिवारमे सभसँ पैघ छलाह हुनका एकटा बहिन और दूटाभाय छलनि', 'input_ids': [11976, 97, 48077, 11976, 108, 48077, 11976, 114, 11976, 224, 11976, 243, 11976, 108, 28225, 105, 11976, 224, 11976, 100, 24231, 233, 11976, 103, 48077, 11976, 100, 24231, 235, 11976, 107, 48077, 11976, 107, 11976, 243, 28225, 250, 11976, 101, 24231, 235, 11976, 106, 220, 28225, 250, 24231, 223, 11976, 110, 48077, 11976, 230, 220, 28225, 230, 11976, 106, 24231, 229, 28225, 103, 11976, 114, 24231, 235, 11976, 248, 11976, 123, 11976, 106, 28225, 105, 11976, 224, 11976, 245, 48077, 11976, 110, 11976, 243, 28225, 105, 24231, 222, 11976, 108, 11976, 255, 24231, 224, 11976, 106, 28225, 250, 11976, 123, 11976, 110, 48077, 11976, 243, 28225, 110, 48077, 11976, 255, 11976, 103, 24231, 223, 11976, 108, 28225, 101, 48077, 11976

In [10]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

/home/c3ilab/Documents/Suryansh/MithilaVerse/env/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding(50258, 768)

In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # Simulate a batch size of 8 (2 * 4)
    save_steps=10_000,
    logging_steps=500,
    logging_dir='./logs',
    overwrite_output_dir=True,
)




trainer = Trainer(
    model=model,                     # The model to be trained
    args=training_args,              # Training arguments
    train_dataset=tokenized_data,    # The training dataset
)

In [12]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [13]:
trainer.train()
trainer.save_model("./gpt2-finetuned")

/home/c3ilab/Documents/Suryansh/MithilaVerse/env/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 500/20232 [04:50<3:11:17,  1.72it/s]

{'loss': 0.4551, 'learning_rate': 4.8764333728746545e-05, 'epoch': 0.07}


  5%|▍         | 1000/20232 [09:41<3:04:44,  1.74it/s]

{'loss': 0.338, 'learning_rate': 4.752866745749308e-05, 'epoch': 0.15}


  7%|▋         | 1500/20232 [14:15<2:48:52,  1.85it/s]

...

{'loss': 0.2226, 'learning_rate': 1.8090154211150655e-06, 'epoch': 2.89}


 99%|█████████▉| 20000/20232 [3:13:21<02:16,  1.70it/s]

{'loss': 0.2173, 'learning_rate': 5.733491498616055e-07, 'epoch': 2.97}


100%|██████████| 20232/20232 [3:15:48<00:00,  1.72it/s]


{'train_runtime': 11748.9668, 'train_samples_per_second': 13.776, 'train_steps_per_second': 1.722, 'train_loss': 0.24715846364514166, 'epoch': 3.0}


In [18]:
from transformers import pipeline
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load your fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
model.to('cuda')
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

input_text = "ताराशंकर बंधोपाध्यायक जन्म  जुलाई  भारतक एक भारतीय अभिनेत्री छ ताराशंकर बंधोपाध्यायक जन्म  जुलाई  भारतक एक भारतीय अभिनेत्री छ"  # prompt in Latin script
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# GPU if available
if torch.cuda.is_available():
    input_ids = input_ids.cuda()

# Generate text
output = model.generate(input_ids, max_length=200)  # max_length adjustment
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Text: ", generated_text)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:  ताराशंकर बंधोपाध्यायक जन्म  जुलाई  भारतक एक भारतीय अभिनेत्री छ ताराशंकर बंधोपाध्यायक जन्म  जुलाई  भारतक एक भारतीय अभिनेत्री छी
ताराशंकर बंधोपाध्यायक जन्म  जुलाई  भारतक एक भारतीय अभिनेत्री छ ताराशंकर बंधोपाध्यायक जन्म  जुलाई  भारतक एक भारतीय अभिनेत्री छी


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhinavmaithil/maithiliwikidump202009uncleaned")

print("Path to dataset files:", path)

In [ ]:
pip install kagglehub